# Welcome to Jupyter Notebooks on the Intel AI DevCloud! 
This document covers the basics of the Jupyter Notebook access to the Intel AI DevCloud. It is not a tutorial on the Jupyter Notebook itself. Rather, we will run through a few examples of how to use the computational resources available on the DevCloud *beyond* the notebook.

The diagram below illustrates the high-level organization of the DevCloud. This tutorial explains how to navigate this organization. 

<img src="https://access.colfaxresearch.com/images/cluster-jn-organization.svg" style="max-width:600px;" />


## Service Terms

By using the Intel AI DevCloud, you are agreeing to the following Service Terms: <br />
<a href="https://access.colfaxresearch.com/doc/Colfax_Cluster_Service_Terms.pdf">https://access.colfaxresearch.com/doc/Colfax_Cluster_Service_Terms.pdf</a>

# 1. 画像データをダウンロードする
どんな画像でもいいですが、とりあえず適当に一枚ダウンロードしておきます。

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/f/f0/Intel_Pentium_II_Overdrive_Engineering_Sample.jpg
!mv Intel_Pentium_II_Overdrive_Engineering_Sample.jpg sample.jpg

# 2. モデルファイルをOpenVINOのIR形式に変換する

ここで使っているモデルファイル「model.pb」は[こちら](https://github.com/hiouchiy/IntelAI_and_Cloud/blob/master/Azure/demo1/model.pb)からダウンロードできます。

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model model.pb \
--data_type FP32 \
--output_dir . \
--input_shape [1,224,224,3]

# 3. バックエンドにてIRを実行するベンチマークスクリプトをビルド
ここでは、OpenVINOの公式？ベンチマークツールを使います。なので、一旦OpenVINOのGit RepoをCloneしてください。

In [ ]:
!git clone https://github.com/opencv/dldt.git

次に、Jupyter notebookのHomeからTerminalを開いていただき、下記の手順にてBuildください。

- cd dldt/inference-engine/samples/
- chmod 755 ./build_samples.sh
- ./build_samples.sh

# 4. バックエンドで動作するShell Scriptを作成

In [ ]:
%%writefile run.sh

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
while getopts 'i:s:r:?' OPTION; do
    case "$OPTION" in
    i)
        NUM_ITER=$OPTARG
        echo "using results base $OPTARG"
      ;;
    s)
        NUM_STREAMS=$OPTARG
        echo "using results base $OPTARG"
      ;;
    r)
        NUM_INFER_REQS=$OPTARG
        echo "running $OPTARG inference requests"
      ;;
    esac  
done


cd $PBS_O_WORKDIR
#cd dldt/inference-engine/tools/benchmark_tool
cd inference_engine_samples_samples_build/intel64/Release

echo "* Hello world from compute server `hostname`!"
echo "* The current directory is ${PWD}."
echo "* Compute server's CPU model and number of logical CPUs:"
lscpu
echo "* Python available to us:"
which python
python --version

#Setup related environment variables
source /opt/intel/openvino/bin/setupvars.sh

#Install related python libraries
pip install networkx defusedxml protobuf py-cpuinfo numpy progress --user

#Run python script in parallel
#numactl -l python benchmark_app.py -m $PBS_O_WORKDIR/model.xml -i $PBS_O_WORKDIR/sample.jpg -d CPU -niter $NUM_ITER -nireq $NUM_INFER_REQS -nstreams $NUM_STREAMS
./benchmark_app -m $PBS_O_WORKDIR/model.badep300_vgg7_32.xml -i $PBS_O_WORKDIR/sample.jpg -d CPU -niter $NUM_ITER -nireq $NUM_INFER_REQS -nstreams $NUM_STREAMS

echo "*Bye"
# Remember to have an empty line at the end of the file; otherwise the last command will not run


# 5. Job Submit

In [ ]:
#Skylakeを使う場合はこちら
#!qsub run.sh -l nodes=1:fpga_compile:ppn=2 -F "-i 100 -s 1 -r 1"

#Cascade Lakeを使う場合はこちら
!qsub run.sh -l nodes=1:experimental:ppn=2 -F "-i 100 -s 4 -r 4"

JobのStatusは下記コマンドにて確認

In [ ]:
!qstat

Submit済みのJobを消す場合は下記コマンド

In [ ]:
!qdel JOB_ID

# 6. Jobの結果を確認

標準出力はoから始まる名前のファイル

In [ ]:
%cat run.sh.o*

エラー出力はeから始まる名前のファイル

In [ ]:
%cat run.sh.e*

# 以降はおまけ

In [ ]:
!rm run.sh.*

In [ ]:
!echo "* How many compute servers are available?"
!pbsnodes | grep "^c" | wc -l

!echo "* How many of them are free?"
!pbsnodes | grep "state = free" | wc -l

!echo "* What are the time limits for queued jobs?"
!qmgr -c 'p q batch' | grep walltime

!echo "* What is the configuration of the available compute servers?"
!pbsnodes | grep properties | sort | uniq